# Trabalho 1 de Segurança Computacional


O trabalho e o código estão disponíveis neste [repositório](https://github.com/marqueswill/seguranca)

Identificação:
- *Aluno:* Willyan Marques de Melo 
- *Matrícula:* 221020940
- *Professor(a):* Lorena de Soza Bezerra Borges
- *Turma:* CIC0201 - Turma 2


## Introdução

O presente trabalho apresenta a cifra Data Encryption Standard (DES) e a implementação da sua versão simplificada (Simplified DES ou S-DES) como forma de exemplificação do algoritmo original. 

### O que é o DES?
   
O DES é um algoritmo clássico de criptografia foi criado pelo atual NIST para o governo dos EUA na década de 1970 como um novo padrão de criptografia proposto pela IBM. Esse algoritmo foi feito com base na Cifra de Feistel e foi amplamente utilizados nas décadas seguintes, até se tornar obsoleto por conta dos avanços tecnológicos e de criptoanálise. O DES foi quebrado em 1999 e, embora outras versões desse algortimo tenham surgido como alternativa de uso, como o 3DES (triple DES), por conta disso entrou em desuso em favor de cifras mais modernas e seguranças como o Advanced Encryption Algorithm (AES).

### A Cifra de Feistel
   
A essência do DES é a cifra de Feistel, que consiste em 16 rodadas de funções de permutação e substituição para incrementar a confusão e a difusão do ciphertext. Cada rodada possui sua própria subchave gerada a partir da chave inicial do algoritmo. Além disso, a cada iteração as metades são trocadas e embaralhas utilizando o resultado do round anterior.

### Overview do DES

O DES é subdivido em 4 partes principais:
   1. Permutação inicial
   2. Rodadas de Feistel
   3. Swap dos bits
   4. Permutação Final

Cada parte do algoritmo principal e da cifra de Feistel serão explicados mais profundamente nas próximas seções


## Python e valores binários
O algotimo apresentado no relatório foi implementado utilizando-se a linguagem Python e, embora o python tenha suporte para valores binários, durante o desenvolvimento das cifras surgiu-se a necessidade de padronizar a representação dos valores em binário do código para que pudessemos manter um controle mais preciso do número de bits. Além disso, aqui também foram adicionadas funções auxiliares que precisaram ser implementadas ao decorrer do projeto, já que nenhuma biblioteca externa foi utilizada.

In [57]:
from typing import List


# Representa um número binário com tamanho fixo
class Bin:
    def __init__(self, value: int, size: int):
        self.value = value  # Valor inteiro
        self.size = size  # Tamanho em bits

    def __repr__(self):
        # Representação para debug (em hexadecimal)
        return f"Bin(value={self.to_hex()}, size={self.size})"

    def __str__(self):
        # Representação em string (em hexadecimal)
        return self.to_hex()

    def __eq__(self, other):
        # Verifica igualdade com outro objeto Bin
        if isinstance(other, Bin):
            return self.value == other.value and self.size == other.size
        return False

    def to_bin(self):
        # Retorna representação binária com zeros à esquerda
        return f"0b{self.value:0{self.size}b}"

    def to_hex(self):
        # Retorna representação hexadecimal com zeros à esquerda
        hex_digits = (self.size + 3) // 4
        return f"0x{self.value:0{hex_digits}X}"


# Extrai bits com base em uma tabela de posições e retorna como novo Bin
def extract_bits(bits: Bin, table: List[List[int]]) -> Bin:
    extracted_bits = []
    for row in table:
        for bit_pos in row:
            bit = (bits.value >> (bits.size - bit_pos)) & 1
            extracted_bits.append(bit)
    result = 0
    for bit in extracted_bits:
        result = (result << 1) | bit
    return Bin(result, len(extracted_bits))


# Divide o valor Bin em duas metades e retorna como dois objetos Bin
def halve(bits: Bin):
    half = (bits.size + 1) // 2
    left = (bits.value >> half) & ((1 << half) - 1)
    right = bits.value & ((1 << half) - 1)
    return Bin(left, half), Bin(right, half)


# Divide o Bin em pedaços menores de tamanho fixo
def split_bits(bits: Bin, chunk_size: int):
    chunks: List[Bin] = []
    for i in range(0, bits.size, chunk_size):
        chunk_value = (bits.value >> (bits.size - i - chunk_size)) & (
            (1 << chunk_size) - 1
        )
        chunks.append(Bin(chunk_value, chunk_size))
    return chunks


# Junta vários objetos Bin em um só
def fuse_bits(*bins: Bin):
    total_value = 0
    total_size = 0
    for b in bins:
        total_value = (total_value << b.size) | b.value
        total_size += b.size
    return Bin(total_value, total_size)


# Troca as metades esquerda e direita de um Bin
def swap_halves(bits: Bin):
    left, right = halve(bits)
    return fuse_bits(right, left)


# Realiza rotação circular para a esquerda
def circular_left_shift(bits: Bin, shift_value: int):
    shift_value %= bits.size
    discarded = bits.value >> (bits.size - shift_value)
    mask = (1 << bits.size) - 1
    shifted = bits.value << shift_value & mask
    return Bin(shifted | discarded, bits.size)


# Realiza operação XOR bit a bit entre dois Bin
def xor(bits1: Bin, bits2: Bin):
    return Bin(bits1.value ^ bits2.value, bits1.size)

## Caixas de Permutação 
Abaixo segueem as tabelas de permutação utilizadas para o DES (initial_perm, final_perm), para geração das subchaves (per1, per2, shift_table) e para as rodadas de feistel (dbox, pbox, sbox).
Essas tabelas são utilizadas como entrada para a função `extract_bits`, que extrai os bits das posições indicadas nas tabelas e retorn um valor binário deles.

In [58]:
# Initial permutation
initial_perm = [
    [2, 6, 3, 1, 4, 8, 5, 7],
]

# Final permutation
final_perm = [
    [4, 1, 3, 5, 7, 2, 8, 6],
]

# Permutade choice 1
per1 = [
    [3, 5, 2, 7, 4, 10, 1, 9, 8, 6],
]

# Permutade choice 2
per2 = [
    [6, 3, 7, 4, 8, 5, 10, 9],
]

# Circular left shift table
shift_table = [1, 2]

# Expansion permutation
dbox = [
    [4, 1, 2, 3, 2, 3, 4, 1],
]

# Transposition permutation
pbox = [
    [2, 4, 3, 1],
]

# Substitution permutation
sbox = [
    [
        [1, 0, 3, 2],
        [3, 2, 1, 0],
        [0, 2, 1, 3],
        [3, 1, 3, 2],
    ],
    [
        [0, 1, 2, 3],
        [2, 0, 1, 3],
        [3, 0, 1, 0],
        [2, 1, 0, 3],
    ],
]

## Cifra de Feistel

### Geração das subchaves
Como a geração das subchaves depende somente da chave original da cifra, podemos modularizar a função `generate_subkeys` do código principal.

In [59]:
# Gera as subchaves a partir da chave principal usando Permuted Choice 1, rotações e Permuted Choice 2
def generate_subkeys(key: Bin):
    pc1 = permuted_choice_1(key)  # Aplica PC-1 para reduzir e permutar a chave original
    left, right = halve(pc1)  # Divide a chave em duas metades

    subkeys: List[Bin] = []
    for i in range(2):  # Executa duas rodadas (pode ser 16 no DES completo)
        left = circular_left_shift(left, shift_table[i])  # Rotaciona a metade esquerda
        right = circular_left_shift(right, shift_table[i])  # Rotaciona a metade direita
        round_key = permuted_choice_2(
            left, right
        )  # Aplica PC-2 para gerar a subchave da rodada
        subkeys.append(round_key)  # Adiciona subchave à lista

    return subkeys


# Aplica a permutação PC-1 na chave original
def permuted_choice_1(key: Bin):
    return extract_bits(key, per1)


# Junta as metades esquerda e direita e aplica PC-2 para gerar a subchave final da rodada
def permuted_choice_2(left: Bin, right: Bin):
    bits = fuse_bits(left, right)
    return extract_bits(bits, per2)

### Rodadas de Feistel

In [ ]:
# Realiza a cifra de Feistel sobre o plaintext usando a chave fornecida
def feistel_encrypt(plaintext: Bin, key: Bin, test=False, reverse=False):
    # Gera as subchaves a partir da chave original
    subkeys = generate_subkeys(key)  
    
    # Inverte as subchaves para decifração
    if reverse:
        subkeys.reverse()  

    round_data: List[List[Bin]] = (
        []
    ) # Armazena os dados de cada rodada (debug/teste)
    
    
    left, right = halve(plaintext)         # Divide o bloco de entrada em duas metades
    for i in range(2):                     
        new_left = right                   # A nova esquerda recebe o valor da direita anterior
        sk = subkeys[i]                    # Subchave da rodada
        magled_right = mangler(right, sk)  # Aplica a função de mistura (mangler)
        new_right = xor(
            magled_right, left
        )                                  # XOR da saída da mangler com a esquerda anterior

        left = new_left                      # Atualiza esquerda
        right = new_right                    # Atualiza direita
        round_data.append([left, right, sk]) # Salva os dados da rodada (opcional)

    # Junta esquerda e direita para formar o bloco final
    result = fuse_bits(left, right)  
    return (result, round_data) if test else result  # Retorna dados extras se test=True


# Função de mistura usada em cada rodada da cifra de Feistel
def mangler(bits: Bin, round_subkey: Bin):
    expanded_right = dbox_expansion(bits)           # Expande os bits com D-box
    xored_right = xor(expanded_right, round_subkey) # Aplica XOR com a subchave
    chunks = split_bits(xored_right, 4)             # Divide em blocos de 4 bits
    fused = sbox_substitution(chunks)               # Aplica substituição usando S-box
    result = pbox_permutation(fused)                # Permuta o resultado com P-box
    return result


# Aplica a expansão dos bits de entrada conforme a D-box
def dbox_expansion(bits: Bin):
    return extract_bits(bits, dbox)


# Substitui blocos de 4 bits usando as S-boxes
def sbox_substitution(chunks: List[Bin]):
    result = []
    for round, chunk in enumerate(chunks):
        line = extract_bits(chunk, [[1, 4]])                # Usa bits 1 e 4 para linha
        column = extract_bits(chunk, [[2, 3]])              # Usa bits 2 e 3 para coluna
        sbox_value = sbox[round][line.value][column.value]  # Busca valor na S-box
        result.append(Bin(sbox_value, 2))                   # Valor da S-box tem 2 bits
    return fuse_bits(*result)                               # Junta os blocos substituídos


# Aplica a permutação final usando a P-box
def pbox_permutation(bits: Bin):
    return extract_bits(bits, pbox)

In [ ]:
# Realiza a cifra completa do DES (versão simplificada)
def des_encryption(plaintext: Bin, key: Bin):
    ciphertext1 = initial_permutation(plaintext)    # Aplica a permutação inicial (IP)
    ciphertext2 = feistel_encrypt(ciphertext1, key) # Executa a crifra de Feistel com a chave
    ciphertext3 = swap_halves(ciphertext2)          # Troca as metades
    ciphertext4 = final_permutation(ciphertext3)    # Aplica a permutação final (IP⁻¹)
    return ciphertext4


# Realiza a decifra completa do DES (versão simplificada)
def des_decryption(plaintext: Bin, key: Bin):
    ciphertext1 = initial_permutation(plaintext)                  # Aplica a permutação inicial (IP)
    ciphertext2 = feistel_encrypt(ciphertext1, key, reverse=True) # Processo de Feistel com subchaves invertidas
    ciphertext3 = swap_halves(ciphertext2)                        # Troca as metades
    ciphertext4 = final_permutation(ciphertext3)                  # Aplica a permutação final (IP⁻¹)
    return ciphertext4


# Aplica a permutação inicial do DES
def initial_permutation(plaintext: Bin):
    return extract_bits(plaintext, initial_perm)


# Aplica a permutação final do DES
def final_permutation(ciphertext: Bin):
    return extract_bits(ciphertext, final_perm)

In [62]:
def test_subkey_generation(example_key):
    try:
        expected_keys = [
            Bin(0b10100100, 8),
            Bin(0b01000011, 8),
        ]

        generated_keys = generate_subkeys(example_key)

    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING SUBKEYS GENERATION".center(40))
    print("-" * 40)
    print("EXPECTED KEY   | GENERATED KEY")
    for g1, g2 in zip(expected_keys, generated_keys):
        print(f"{g1.to_bin():<14} | {g2.to_bin()}")

    all_match = all(g1 == g2 for g1, g2 in zip(expected_keys, generated_keys))
    if all_match:
        print("\nTest passed: All subkeys match expected values.")
    else:
        print("\nTest failed: Subkeys do not match expected values.")


def test_feistel_encrypt(example_plaintext, example_key):
    try:
        expected_rounds = [
            [Bin(0b1101, 4), Bin(0b1010, 4), Bin(0b10100100, 8)],
            [Bin(0b1010, 4), Bin(0b0010, 4), Bin(0b01000011, 8)],
        ]

        initial_permutation = extract_bits(example_plaintext, initial_perm)
        example_key = Bin(0b1010000010, 10)
        rounds_data = feistel_encrypt(initial_permutation, example_key, test=True)[1]

    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING FEISTEL ENCRYPTION".center(40))
    print("-" * 40)

    print("EXPECTED ROUND VALUES")
    print("LEFT       | RIGHT      | SUBKEY")
    for l, r, sk in expected_rounds:
        print(f"{l.to_bin():<10} | {r.to_bin():<10} | {sk.to_bin()}")

    print("\nGENERATED ROUND VALUES")
    print("LEFT       | RIGHT      | SUBKEY")
    for l, r, sk in rounds_data:
        print(f"{l.to_bin():<10} | {r.to_bin():<10} | {sk.to_bin()}")

    print()
    all_match = all(g1 == g2 for g1, g2 in zip(expected_rounds, rounds_data))
    if all_match:
        print("Test passed: All rounds match expected values.")
    else:
        print("Test failed: Rounds do not match expected values.")


def test_des_encryption(
    example_plaintext: Bin, example_key: Bin, expected_ciphertext: Bin
):
    try:
        generated_ciphertext = des_encryption(example_plaintext, example_key)
    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING DES ENCRYPTION".center(40))
    print("-" * 40)

    print(f"{'Plaintext:':<22}{example_plaintext.to_bin()}")
    print(f"{'Key:':<22}{example_key.to_bin()}")
    print(f"{'Expected Ciphertext:':<22}{expected_ciphertext.to_bin()}")
    print(f"{'Obtained Ciphertext:':<22}{generated_ciphertext.to_bin()}\n")

    assert (
        generated_ciphertext == expected_ciphertext
    ), "Test failed: Ciphertext does not match expected value."

    print("Test passed: Ciphertext matches expected value.")


def test_des_decryption(
    example_ciphertext: Bin, example_key: Bin, expected_plaintext: Bin
):
    try:
        generated_plaintext = des_decryption(example_ciphertext, example_key)

    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING DES DECRYPTION".center(40))
    print("-" * 40)

    print(f"{'Ciphertext:':<22}{example_ciphertext.to_bin()}")
    print(f"{'Key:':<22}{example_key.to_bin()}")
    print(f"{'Expected Plaintext:':<22}{expected_plaintext.to_bin()}")
    print(f"{'Obtained Plaintext:':<22}{generated_plaintext.to_bin()}\n")

    assert (
        generated_plaintext == expected_plaintext
    ), "Test failed: Ciphertext does not match expected value."

    print("Test passed: Ciphertext matches expected value.")

In [63]:
example_plaintext2 = Bin(0b10010111 , 8)
example_key2 = Bin(0b1010000010, 10)
expected_ciphertext2 = Bin(0b00111000 , 8)

test_subkey_generation(example_key2)
test_feistel_encrypt(example_plaintext2, example_key2)
test_des_encryption(example_plaintext2, example_key2, expected_ciphertext2)
test_des_decryption(expected_ciphertext2, example_key2, example_plaintext2)

      

TESTING SUBKEYS GENERATION      
----------------------------------------
EXPECTED KEY   | GENERATED KEY
0b10100100     | 0b10100100
0b01000011     | 0b01000011

Test passed: All subkeys match expected values.
      

TESTING FEISTEL ENCRYPTION      
----------------------------------------
EXPECTED ROUND VALUES
LEFT       | RIGHT      | SUBKEY
0b1101     | 0b1010     | 0b10100100
0b1010     | 0b0010     | 0b01000011

GENERATED ROUND VALUES
LEFT       | RIGHT      | SUBKEY
0b1101     | 0b1010     | 0b10100100
0b1010     | 0b0010     | 0b01000011

Test passed: All rounds match expected values.
        

TESTING DES ENCRYPTION        
----------------------------------------
Plaintext:            0b10010111
Key:                  0b1010000010
Expected Ciphertext:  0b00111000
Obtained Ciphertext:  0b00111000

Test passed: Ciphertext matches expected value.
        

TESTING DES DECRYPTION        
----------------------------------------
Ciphertext:           0b00111000
Key:        

Referências

https://www.geeksforgeeks.org/data-encryption-standard-des-set-1/

https://www.geeksforgeeks.org/simplified-data-encryption-standard-key-generation/

https://www.geeksforgeeks.org/simplified-data-encryption-standard-set-2/

https://www.geeksforgeeks.org/shift-micro-operations-in-computer-architecture/

https://aprender3.unb.br/pluginfile.php/3076851/mod_resource/content/1/Aula%2008%20-%20Cifra%20de%20Bloco%20-%20DES%20%282%29.pdf